# Crypto AI Backtest (Multi-Coin)

This notebook trains an AI model on BTC then applies it to BTC, GALA and XRP. It backtests a Bollinger+AI-filtered strategy on each coin and shows performance metrics and equity curves.

**Run in Google Colab or Jupyter.** Uncomment the pip installs if needed.

In [ ]:
# Install packages if running in a fresh environment
# !pip install yfinance pandas numpy ta scikit-learn joblib vectorbt matplotlib


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import ta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib
import vectorbt as vbt
import matplotlib.pyplot as plt

pd.options.display.max_columns = 100


## Step 1 — Download 30m OHLCV data for BTC, GALA, XRP (180 days)

In [ ]:
symbols = ['BTC-USD', 'GALA-USD', 'XRP-USD']
interval = '30m'
period = '180d'

price_data = {}
for sym in symbols:
    print(f'Downloading {sym}...')
    df = yf.download(sym, period=period, interval=interval, progress=False)
    if df.empty:
        print(f'Warning: no data for {sym}')
    price_data[sym] = df.dropna()

# show samples
for s, df in price_data.items():
    print(s, 'rows:', len(df))
    display(df.head())


## Step 2 — Feature engineering function (RSI, MACD, Bollinger, volume change, candlestick features)

In [ ]:
def build_features(df):
    df = df.copy()
    # Indicators
    df['rsi'] = ta.momentum.RSIIndicator(df['Close']).rsi()
    macd = ta.trend.MACD(df['Close'])
    df['macd'] = macd.macd()
    df['macd_signal'] = macd.macd_signal()
    bb = ta.volatility.BollingerBands(df['Close'])
    df['bb_mid'] = bb.bollinger_mavg()
    df['bb_high'] = bb.bollinger_hband()
    df['bb_low'] = bb.bollinger_lband()
    df['bb_width'] = (df['bb_high'] - df['bb_low']) / df['bb_mid']
    df['percent_b'] = (df['Close'] - df['bb_low']) / (df['bb_high'] - df['bb_low'])
    df['volume_change'] = df['Volume'].pct_change()

    # Candlestick features: shooting star & hammer simple heuristics
    o = df['Open']; h = df['High']; l = df['Low']; c = df['Close']
    body = abs(c - o)
    candle_range = h - l
    upper_shadow = h - np.maximum(o, c)
    lower_shadow = np.minimum(o, c) - l
    df['shooting_star'] = ((body <= 0.3*candle_range) & (upper_shadow >= 2*body) & (lower_shadow <= 0.2*body)).astype(int)
    df['hammer'] = ((body <= 0.3*candle_range) & (lower_shadow >= 2*body) & (upper_shadow <= 0.2*body)).astype(int)

    return df.dropna()

# Build features for all symbols
features = {sym: build_features(df) for sym, df in price_data.items()}
for s in features:
    print(s, 'features rows:', len(features[s]))
    display(features[s].head())


## Step 3 — Label data for training

Label definition: a future horizon of 3 bars (~90 minutes). Label = 1 if future return > 0.2% (0.002), else 0. You can tune horizon & threshold.

In [ ]:
horizon = 3
label_threshold = 0.002
labeled = {}
for sym, df in features.items():
    df2 = df.copy()
    df2['future_return'] = df2['Close'].shift(-horizon) / df2['Close'] - 1
    df2 = df2.dropna()
    df2['label'] = (df2['future_return'] > label_threshold).astype(int)
    labeled[sym] = df2
    print(sym, 'label distribution:')
    print(df2['label'].value_counts(normalize=True).to_string())

# Use BTC for training by default
train_sym = 'BTC-USD'
train_df = labeled[train_sym]
train_df[['Close','future_return','label']].head()


## Step 4 — Prepare features and train RandomForest

We use a modest feature set. Train on BTC and save model.

In [ ]:
feature_cols = ['rsi','macd','bb_mid','bb_high','bb_low','bb_width','percent_b','volume_change','shooting_star','hammer']
X = train_df[feature_cols].fillna(0)
y = train_df['label']

# Train-test split (time-series aware: no shuffle)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

model = RandomForestClassifier(n_estimators=200, max_depth=8, random_state=42)
model.fit(X_train, y_train)

print('Training complete')
from sklearn.metrics import classification_report
print(classification_report(y_test, model.predict(X_test)))

# Save model
joblib.dump(model, 'crypto_ai_model.pkl')
print('Model saved to crypto_ai_model.pkl')


## Step 5 — Apply model to all symbols and backtest

We will generate AI signals (1=approve trade) and then backtest Bollinger entries filtered by AI approval (entry only when price <= lower band and ai==1). Exits are price >= upper band.

In [ ]:
results = {}
for sym in symbols:
    print('\nProcessing', sym)
    df = labeled[sym].copy()
    # Align features
    X_sym = df[feature_cols].fillna(0)
    df['ai_signal'] = model.predict(X_sym)
    # Bollinger rule
    entries = (df['Close'] <= df['bb_low']) & (df['ai_signal'] == 1)
    exits = (df['Close'] >= df['bb_high'])
    # Shift execution to next bar
    entries_exec = entries.shift(1).fillna(False)
    exits_exec = exits.shift(1).fillna(False)

    pf = vbt.Portfolio.from_signals(df['Close'], entries_exec, exits_exec,
                                    init_cash=10000, fees=0.001, slippage=0.0005, freq='30T')
    stats = pf.stats()
    print(f'--- Stats for {sym} ---')
    display(stats)
    results[sym] = pf


## Step 6 — Plot equity curves for all symbols

In [ ]:
plt.figure(figsize=(12,6))
for sym, pf in results.items():
    (pf.value() / pf.value().iloc[0]).vbt.plot(label=sym)
plt.title('Normalized Equity Curves (Bollinger + AI filter)')
plt.legend()
plt.show()

# Also show individual detailed plots if desired
for sym, pf in results.items():
    print('\nDetailed plot for', sym)
    display(pf.stats())
    pf.plot().show()


## Next steps

- Tune label threshold, horizon, model hyperparameters.
- Consider training on combined multi-coin dataset for better generalization.
- Add transaction costs & more realistic slippage models.
- Run walk-forward validation and cross-validation for robustness.
